# Instalações

In [1]:
# @title Instalar
%pip install cplex
%pip install docplex
%pip install tensorflow
%pip install matplotlib
from IPython.display import clear_output 
clear_output()

# Importações

In [2]:
# @title Importar
import os
import cplex
import numpy as np
import pandas as pd
from time import time
import tensorflow as tf
from cplex import infinity
import docplex.mp.model as mp
from typing import List, Tuple, Any
from dataclasses import dataclass
from docplex.mp.constr import LinearConstraint
# import matplotlib.pyplot as plt
# from statistics import mean, stdev
# import matplotlib.patches as patches

# Drive

In [3]:
# @title Montar o Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# base_path = '/content/drive/My Drive/Colab Notebooks'
base_path = './'

In [4]:
# @title Caminho para os arquivos no Google Drive
datasets_path = f'{base_path}/datasets'
# resultados_path = f'{base_path}/resultados'

# Milp

## Fischetti

In [5]:
# @title Original
def codify_network_fischetti(
    mdl: mp.Model,
    layers,
    input_variables,
    auxiliary_variables,
    intermediate_variables,
    decision_variables,
    output_variables,
):
    output_bounds = []
    bounds = []

    for i in range(len(layers)):
        A = layers[i].get_weights()[0].T
        b = layers[i].bias.numpy()
        x = input_variables if i == 0 else intermediate_variables[i - 1]
        if i != len(layers) - 1:
            s = auxiliary_variables[i]
            a = decision_variables[i]
            y = intermediate_variables[i]
        else:
            y = output_variables

        for j in range(A.shape[0]):
            if i != len(layers) - 1:
                mdl.add_constraint(
                    A[j, :] @ x + b[j] == y[j] - s[j], ctname=f"c_{i}_{j}"
                )
                mdl.add_indicator(a[j], y[j] <= 0, 1)
                mdl.add_indicator(a[j], s[j] <= 0, 0)

                mdl.maximize(y[j])
                mdl.solve()
                ub_y = mdl.solution.get_objective_value()
                mdl.remove_objective()

                mdl.maximize(s[j])
                mdl.solve()
                ub_s = mdl.solution.get_objective_value()
                mdl.remove_objective()

                y[j].set_ub(ub_y)
                s[j].set_ub(ub_s)

                bounds.append([ub_s, ub_y])

            else:
                mdl.add_constraint(A[j, :] @ x + b[j] == y[j], ctname=f"c_{i}_{j}")
                mdl.maximize(y[j])
                mdl.solve()
                ub = mdl.solution.get_objective_value()
                mdl.remove_objective()

                mdl.minimize(y[j])
                mdl.solve()
                lb = mdl.solution.get_objective_value()
                mdl.remove_objective()

                y[j].set_lb(lb)
                y[j].set_ub(ub)

                output_bounds.append([lb, ub])

                bounds.append([lb, ub])

    return mdl, output_bounds, bounds

In [6]:
# @title Relaxado
def codify_network_fischetti_relaxed(
    mdl,
    mdl_original,
    layers,
    input_variables,
    auxiliary_variables,
    intermediate_variables,
    decision_variables,
    output_variables,
    output_bounds_binary_variables,
    bounds=[]
):
    output_bounds = []
    

    for i in range(len(layers)):  # para cada camada
        A = layers[i].get_weights()[0].T
        b = layers[i].bias.numpy()
        x = input_variables if i == 0 else intermediate_variables[i - 1]
        if i != len(layers) - 1:
            s = auxiliary_variables[i]
            a = decision_variables[i]
            y = intermediate_variables[i]
        else:
            y = output_variables

        for j in range(A.shape[0]):  # para cada neuronio da camada
            if i != len(layers) - 1:  # se não for a última camada(camada de saída)
                s_ub = mdl_original.get_var_by_name(f's_{i}_{j}').ub
                y_ub = mdl_original.get_var_by_name(f'y_{i}_{j}').ub
                
                m_less = -s_ub  # L
                m_more = y_ub   # U
                
                y[j].set_lb(max(0, m_less))
                s[j].set_lb(max(0, -m_more))
                
                y[j].set_ub(max(0, m_more))
                s[j].set_ub(max(0, -m_less))

                # mdl.add_constraint(y[j] <= (m_more * ( A[j, :] @ x + b[j] - m_less) ) / (m_more - m_less) )
                # mdl.add_constraint(y[j] >= A[j, :] @ x + b[j] )
                
                if m_more <= 0:
                    mdl.add_constraint(y[j] == 0)
                    continue

                if m_less >= 0:
                    mdl.add_constraint(A[j, :] @ x + b[j] == y[j])
                    continue

                if m_less < 0 and 0 < m_more:
                    mdl.add_constraint(A[j, :] @ x + b[j] == y[j] - s[j], ctname=f"c_{i}_{j}")
                    # mdl.add_constraint(y[j] <= m_more * (1 - a[j]))
                    # mdl.add_constraint(s[j] <= -m_less * a[j])
                    mdl.add_constraint(y[j] <= m_more * a[j])
                    mdl.add_constraint(s[j] <= -m_less * (1 - a[j]))
                    continue
            

            else:
                mdl.add_constraint(A[j, :] @ x + b[j] == y[j], ctname=f"c_{i}_{j}")
                lb, ub = output_bounds_binary_variables[j]
                y[j].set_lb(lb)
                y[j].set_ub(ub)
                output_bounds.append([lb, ub])

    return mdl, output_bounds

In [7]:
# @title Tjeng
def codify_network_tjeng(
    mdl,
    layers,
    input_variables,
    intermediate_variables,
    decision_variables,
    output_variables,
):
    output_bounds = []

    for i in range(len(layers)):
        A = layers[i].get_weights()[0].T
        b = layers[i].bias.numpy()
        x = input_variables if i == 0 else intermediate_variables[i - 1]
        if i != len(layers) - 1:
            a = decision_variables[i]
            y = intermediate_variables[i]
        else:
            y = output_variables

        for j in range(A.shape[0]):
            mdl.maximize(A[j, :] @ x + b[j])
            mdl.solve()
            ub = mdl.solution.get_objective_value()
            mdl.remove_objective()

            if ub <= 0 and i != len(layers) - 1:
                # print("ENTROU, o ub é negativo, logo y = 0")
                mdl.add_constraint(y[j] == 0, ctname=f"c_{i}_{j}")
                continue

            mdl.minimize(A[j, :] @ x + b[j])
            mdl.solve()
            lb = mdl.solution.get_objective_value()
            mdl.remove_objective()

            if lb >= 0 and i != len(layers) - 1:
                # print("ENTROU, o lb >= 0, logo y = Wx + b")
                mdl.add_constraint(A[j, :] @ x + b[j] ==
                                   y[j], ctname=f"c_{i}_{j}")
                continue

            if i != len(layers) - 1:
                mdl.add_constraint(y[j] <= A[j, :] @ x +
                                   b[j] - lb * (1 - a[j]))
                mdl.add_constraint(y[j] >= A[j, :] @ x + b[j])
                mdl.add_constraint(y[j] <= ub * a[j])

                # mdl.maximize(y[j])
                # mdl.solve()
                # ub_y = mdl.solution.get_objective_value()
                # mdl.remove_objective()
                # y[j].set_ub(ub_y)

            else:
                mdl.add_constraint(A[j, :] @ x + b[j] == y[j])
                # y[j].set_ub(ub)
                # y[j].set_lb(lb)
                output_bounds.append([lb, ub])

    return mdl, output_bounds

In [8]:
# @title Domain and Bounds
def get_domain_and_bounds_inputs(dataframe):
    domain = []
    bounds = []
    for column in dataframe.columns[:-1]:
        if len(dataframe[column].unique()) == 2:
            domain.append("B")
            bound_inf = dataframe[column].min()
            bound_sup = dataframe[column].max()
            bounds.append([bound_inf, bound_sup])
        elif np.any(
            dataframe[column].unique().astype(np.int64)
            != dataframe[column].unique().astype(np.float64)
        ):
            domain.append("C")
            bound_inf = dataframe[column].min()
            bound_sup = dataframe[column].max()
            bounds.append([bound_inf, bound_sup])
        else:
            domain.append("I")
            bound_inf = dataframe[column].min()
            bound_sup = dataframe[column].max()
            bounds.append([bound_inf, bound_sup])

    return domain, bounds

## Codify Network

**X ---- E**

x1 = 1 ∧ x2 = 3 ∧ F ∧ ¬E  
*INSATISFÁTIVEL*

x1 ≥ 0 ∧ x1 ≤ 100 ∧ x2 = 3 ∧ F ∧ ¬E  
*INSATISFÁTIVEL* → x1 não é relevante,  
*SATISFATÍVEL* → x1 é relevante


In [9]:
# @title Original
def codify_network(model, dataframe, method, relaxe_constraints):
    layers = model.layers
    num_features = layers[0].get_weights()[0].shape[0]
    mdl = mp.Model()

    domain_input, bounds_input = get_domain_and_bounds_inputs(dataframe)
    bounds_input = np.array(bounds_input)

    if relaxe_constraints:
        input_variables = mdl.continuous_var_list(
            num_features, lb=bounds_input[:,
                                          0], ub=bounds_input[:, 1], name="x"
        )
    else:
        input_variables = []
        for i in range(len(domain_input)):
            lb, ub = bounds_input[i]
            if domain_input[i] == "C":
                input_variables.append(
                    mdl.continuous_var(lb=lb, ub=ub, name=f"x_{i}"))
            elif domain_input[i] == "I":
                input_variables.append(
                    mdl.integer_var(lb=lb, ub=ub, name=f"x_{i}"))
            elif domain_input[i] == "B":
                input_variables.append(mdl.binary_var(name=f"x_{i}"))

    intermediate_variables = []
    auxiliary_variables = []
    decision_variables = []

    for i in range(len(layers) - 1):
        weights = layers[i].get_weights()[0]
        intermediate_variables.append(
            mdl.continuous_var_list(
                weights.shape[1], lb=0, name="y", key_format=f"_{i}_%s"
            )
        )

        if method == "fischetti":
            auxiliary_variables.append(
                mdl.continuous_var_list(
                    weights.shape[1], lb=0, name="s", key_format=f"_{i}_%s"
                )
            )

        if relaxe_constraints and method == "tjeng":
            decision_variables.append(
                mdl.continuous_var_list(
                    weights.shape[1], name="a", lb=0, ub=1, key_format=f"_{i}_%s"
                )
            )
        else:
            decision_variables.append(
                mdl.binary_var_list(
                    weights.shape[1], name="a", lb=0, ub=1, key_format=f"_{i}_%s"
                )
            )

    output_variables = mdl.continuous_var_list(
        layers[-1].get_weights()[0].shape[1], lb=-infinity, name="o" #type: ignore
    )

    if method == "tjeng":
        mdl, output_bounds = codify_network_tjeng(
            mdl,
            layers,
            input_variables,
            intermediate_variables,
            decision_variables,
            output_variables,
        )
    else:
        mdl, output_bounds, bounds = codify_network_fischetti(
            mdl,
            layers,
            input_variables,
            auxiliary_variables,
            intermediate_variables,
            decision_variables,
            output_variables,
        )

    if relaxe_constraints:
        # Tighten domain of variables 'a'
        for i in decision_variables:
            for a in i:
                a.set_vartype("Integer")

        # Tighten domain of input variables
        for i, x in enumerate(input_variables):
            if domain_input[i] == "I":
                x.set_vartype("Integer")
            elif domain_input[i] == "B":
                x.set_vartype("Binary")
            elif domain_input[i] == "C":
                x.set_vartype("Continuous")

    return mdl, output_bounds, bounds


In [10]:
# @title Relaxado
def codify_network_relaxed(
    model, mdl_original:mp.Model, dataframe:pd.DataFrame, method:str, relaxe_constraints:bool, output_bounds_binary_variables, bounds
):
    
    
    layers = model.layers
    num_features = layers[0].get_weights()[0].shape[0]
    mdl = mp.Model()

    domain_input, bounds_input = get_domain_and_bounds_inputs(dataframe)
    bounds_input = np.array(bounds_input)

    if relaxe_constraints:
        input_variables = mdl.continuous_var_list(
            num_features, lb=bounds_input[:,0], ub=bounds_input[:, 1], name="x"
        )
    else:
        input_variables = []
        for i in range(len(domain_input)):
            lb, ub = bounds_input[i]
            input_variables.append(
                mdl.continuous_var(lb=lb, ub=ub, name=f"x_{i}"))

    intermediate_variables = []
    auxiliary_variables = []
    decision_variables = []

    for i in range(len(layers) - 1):
        weights = layers[i].get_weights()[0]
        intermediate_variables.append(
            mdl.continuous_var_list(
                weights.shape[1], lb=0, name="y", key_format=f"_{i}_%s"
            )
        )
        auxiliary_variables.append(
            mdl.continuous_var_list(
                weights.shape[1], lb=0, name="s", key_format=f"_{i}_%s"
            )
        )
        decision_variables.append(
            mdl.continuous_var_list(
                weights.shape[1], name="a", lb=0, ub=1, key_format=f"_{i}_%s"
            )
        )

    output_variables = mdl.continuous_var_list(
        layers[-1].get_weights()[0].shape[1], lb=-infinity, name="o"
    )

    mdl, output_bounds = codify_network_fischetti_relaxed(
        mdl,
        mdl_original,
        layers,
        input_variables,
        auxiliary_variables,
        intermediate_variables,
        decision_variables,
        output_variables,
        output_bounds_binary_variables,
        bounds=bounds
    )

    
    if relaxe_constraints:
        # Tighten domain of variables 'a'
        for i in decision_variables:
            for a in i:
                a.set_vartype("Continuous")

        # Tighten domain of input variables
        for i, x in enumerate(input_variables):
            if domain_input[i] == "I":
                x.set_vartype("Integer")
            elif domain_input[i] == "B":
                x.set_vartype("Binary")
            elif domain_input[i] == "C":
                x.set_vartype("Continuous")

    return mdl, output_bounds


# Teste

In [11]:
# @title Insert Outputs
def insert_output_constraints_fischetti(
    mdl, output_variables, network_output, binary_variables
):
    variable_output = output_variables[network_output]
    aux_var = 0
    for i, output in enumerate(output_variables):
        if i != network_output:
            p = binary_variables[aux_var]
            aux_var += 1
            mdl.add_indicator(p, variable_output <= output, 1)

    return mdl

def insert_output_constraints_tjeng(
    mdl, output_variables, network_output, binary_variables, output_bounds
):
    variable_output = output_variables[network_output]
    upper_bounds_diffs = (
        output_bounds[network_output][1] - np.array(output_bounds)[:, 0]
    )  # Output i: oi - oj <= u1 = ui - lj
    aux_var = 0

    for i, output in enumerate(output_variables):
        if i != network_output:
            ub = upper_bounds_diffs[i]
            z = binary_variables[aux_var]
            mdl.add_constraint(variable_output - output - ub * (1 - z) <= 0)
            aux_var += 1

    return mdl


## Explicações

**X ---- E**

x1 = 1 ∧ x2 = 3 ∧ F ∧ ¬E  
*INSATISFÁTIVEL*

x1 ≥ 0 ∧ x1 ≤ 100 ∧ x2 = 3 ∧ F ∧ ¬E  
*INSATISFÁTIVEL* → x1 não é relevante,  
*SATISFATÍVEL* → x1 é relevante

In [12]:
# @title Original
def get_minimal_explanation(
    mdl,
    network_input,
    network_output,
    n_classes,
    method,
    output_bounds=None,
    initial_explanation=None,
    min_max_results_path_original = '' 
) -> Tuple[List[LinearConstraint], mp.Model]:
    assert not (
        method == "tjeng" and output_bounds == None
    ), "If the method tjeng is chosen, output_bounds must be passed."

    output_variables = [mdl.get_var_by_name(f"o_{i}") for i in range(n_classes)]

    if initial_explanation is None:
        input_constraints = mdl.add_constraints(
            [
                mdl.get_var_by_name(f"x_{i}") == feature.numpy()
                for i, feature in enumerate(network_input[0])
            ],
            names="input",
        )
    else:
        input_constraints = mdl.add_constraints(
            [
                mdl.get_var_by_name(f"x_{i}") == network_input[0][i].numpy()
                for i in initial_explanation
            ],
            names="input",
        )

    binary_variables = mdl.binary_var_list(n_classes - 1, name="b")
    mdl.add_constraint(mdl.sum(binary_variables) >= 1)

    if method == "tjeng":
        mdl = insert_output_constraints_tjeng(
            mdl, output_variables, network_output, binary_variables, output_bounds
        )
    else:
        mdl = insert_output_constraints_fischetti(
            mdl, output_variables, network_output, binary_variables
        )

    # columns = [ (f'o_{i}_lb', f'o_{i}_ub') for i in range(len(output_variables))]
    # elements_list = [element for tupla in columns for element in tupla]
    # min_max_outputs_df = pd.DataFrame(columns = elements_list)

    for constraint in input_constraints:
        mdl.remove_constraint(constraint)
        mdl.solve(log_output=False)
        # relevante = False
        if mdl.solution is not None:
            # relevante = True
            # if relevante:
            #     antes = []
            #     depois = []
            #     for i in range(len(output_variables)):
            #         o_i = output_variables[i]
            #         lb = o_i.lb
            #         ub = o_i.ub
            #         antes.append((lb, ub))

            #         mdl.minimize(o_i)
            #         sol = mdl.solve()
            #         new_lb = sol.get_objective_value()
            #         mdl.remove_objective()
            #         sol = None

            #         mdl.maximize(o_i)
            #         sol = mdl.solve()
            #         new_ub = sol.get_objective_value()
            #         mdl.remove_objective()
            #         sol = None
                    
            #         depois.append(new_lb)
            #         depois.append(new_ub)
            #     min_max_outputs_df.loc[len(min_max_outputs_df)] = depois
            mdl.add_constraint(constraint)
    
    # min_max_outputs_df.to_csv(f'{min_max_results_path_original}/df.csv')
    inputs = mdl.find_matching_linear_constraints("input")
    return (inputs, mdl)

In [13]:

# @title Relaxado
def get_explanation_relaxed(
    mdl: mp.Model,
    network_input,
    network_output,
    n_classes,
    method,
    output_bounds=None,
    initial_explanation=None,
    delta=0.1,
    min_max_results_path_relaxed_global = ''
) -> Tuple[List[LinearConstraint], mp.Model]:
    assert not (
        method == "tjeng" and output_bounds == None
    ), "If the method tjeng is chosen, output_bounds must be passed."

    output_variables = [mdl.get_var_by_name(f"o_{i}") for i in range(n_classes)]

    if initial_explanation is None:
        input_constraints = mdl.add_constraints(
            [
                mdl.get_var_by_name(f"x_{i}") == feature.numpy()
                for i, feature in enumerate(network_input[0])
            ],
            names="input",
        )
    else:
        input_constraints = mdl.add_constraints(
            [
                mdl.get_var_by_name(f"x_{i}") == network_input[0][i].numpy()
                for i in initial_explanation
            ],
            names="input",
        )
        
    
    binary_variables = mdl.binary_var_list(n_classes - 1, name="b")
    mdl.add_constraint(mdl.sum(binary_variables) >= 1) #type: ignore
    # todo: salvar modelo durante a explicação

    if method == "tjeng":
        mdl = insert_output_constraints_tjeng(
            mdl, output_variables, network_output, binary_variables, output_bounds
        )

    else:
        mdl = insert_output_constraints_fischetti(
            mdl, output_variables, network_output, binary_variables
        )


    x_vars = mdl.find_matching_vars('x_')
    x_values = [feature for i, feature in enumerate(network_input[0])]
    i = 0 
    
    for constraint in input_constraints:
        mdl.remove_constraint(constraint)
        
        x, v = x_vars[i] , float(x_values[i])
        i = i+1
         
        
        left = max(0, v - delta)
        right = min(1, v + delta)
        
        if left > 0:
            constraint_left =   mdl.add_constraint(x >= right)
        if right < 1:
            constraint_right =  mdl.add_constraint(x <=  left)

        
        #constraint_left = mdl.add_constraint(max(0, v - delta) <= x)
        #constraint_right = mdl.add_constraint(x <= min(1, v + delta))

        mdl.solve(log_output=False)
        if mdl.solution is not None:
            mdl.add_constraint(constraint)
            if left > 0:
                mdl.remove_constraint(constraint_left)
            if right < 1:
                mdl.remove_constraint(constraint_right)
            
    
    inputs = mdl.find_matching_linear_constraints("input")
    return (inputs, mdl)

# Gerar Rede Neural

In [14]:
# @title Gerar Rede Neural
def gerar_rede(dir_path: str, num_classes: int, n_neurons: int, n_hidden_layers: int):
    data_train = pd.read_csv(dir_path + "/" + "train.csv").to_numpy()
    data_test = pd.read_csv(dir_path + "/" + "test.csv").to_numpy()

    x_train, y_train = data_train[:, :-1], data_train[:, -1]
    x_test, y_test = data_test[:, :-1], data_test[:, -1]

    y_train_ohe = tf.keras.utils.to_categorical(
        y_train, num_classes=num_classes)
    y_test_ohe = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

    model = tf.keras.Sequential(
        [
            tf.keras.layers.Input(shape=[x_train.shape[1]]),
        ]
    )

    for _ in range(n_hidden_layers):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))

    model.add(tf.keras.layers.Dense(num_classes, activation="softmax"))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    model_path = os.path.join(
        dir_path, "models", f"model_{n_hidden_layers}layers_{n_neurons}neurons_teste.h5"
    )

    es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
    ck = tf.keras.callbacks.ModelCheckpoint(
        model_path, monitor="val_accuracy", save_best_only=True
    )

    start = time()
    model.fit(
        x_train,
        y_train_ohe,
        batch_size=4,
        epochs=100,
        validation_data=(x_test, y_test_ohe),
        verbose=2,
        callbacks=[ck, es],
    )
    print(f"Tempo de Treinamento: {time()-start}")

    # salvar modelo
    model = tf.keras.models.load_model(model_path)

    # avaliar modelo com os dados de treinamento
    print("Resultado Treinamento")
    model.evaluate(x_train, y_train_ohe, verbose=2)

    # avaliar modelo com os dados de teste
    print("Resultado Teste")
    model.evaluate(x_test, y_test_ohe, verbose=2)
    return model

In [15]:
# @title Gerar Rede Neural a partir de um dataset
def gerar_rede_com_dataset_iris(n_neurons=20, n_hidden_layers=1):
    dir_path = f"{base_path}/datasets/iris"
    num_classes = 3
    return gerar_rede(dir_path, num_classes, n_neurons, n_hidden_layers)


def gerar_rede_com_dataset_digits(n_neurons=20, n_hidden_layers=1):
    dir_path = f"{base_path}/datasets/digits"
    num_classes = 10
    return gerar_rede(dir_path, num_classes, n_neurons, n_hidden_layers)


def gerar_rede_com_dataset_wine(n_neurons=20, n_hidden_layers=1):
    dir_path = "datasets\\wine"
    num_classes = 10
    return gerar_rede(dir_path, num_classes, n_neurons, n_hidden_layers)

# Explicar Instância

In [16]:
# @title explain_instance
def explain_instance(
    initial_network,
    configuration: dict,
    instance_index: int,
    data: pd.DataFrame,
    model_h5,
    n_classes,
    min_max_results_path_original = ''
) -> Tuple[List[LinearConstraint], mp.Model, int]:
    method = configuration["method"]
    (
        mdl_milp_with_binary_variable,
        output_bounds_binary_variables,
        bounds,
    ) = initial_network
    network_input = data.iloc[instance_index, :-1]
    # print(network_input)  # network_input = instance
    network_input = tf.reshape(tf.constant(network_input), (1, -1))
    network_output = model_h5.predict(tf.constant(network_input))[0]
    network_output = tf.argmax(network_output)
    mdl_aux = mdl_milp_with_binary_variable.clone()
    (explanation, model) = get_minimal_explanation(
        mdl_aux,
        network_input,
        network_output,
        n_classes=n_classes,
        method=method,
        output_bounds=output_bounds_binary_variables,
        min_max_results_path_original = min_max_results_path_original
    )

    return (explanation, model, network_output)

In [17]:
# @title explain_instance_relaxed
def explain_instance_relaxed(
    initial_network,
    initial_network_relaxed,
    configuration: dict,
    instance_index: int,
    data: pd.DataFrame,
    model_h5,
    n_classes,
    delta=1.0,
    min_max_results_path_relaxed_ = ''
) -> Tuple[List[LinearConstraint], mp.Model]:
    method = configuration["method"]
    (
        mdl_milp_with_binary_variable,
        output_bounds_binary_variables,
        bounds,
    ) = initial_network

    model_milp_relaxed, output_bounds_relaxed = initial_network_relaxed
    network_input = data.iloc[instance_index, :-1]
    # print(network_input)  # network_input = instance
    network_input = tf.reshape(tf.constant(network_input), (1, -1))
    network_output = model_h5.predict(tf.constant(network_input))[0]
    network_output = tf.argmax(network_output)

    mdl_aux = model_milp_relaxed.clone()

    (explanation, model) = get_explanation_relaxed(
        mdl_aux,
        network_input,
        network_output,
        n_classes=n_classes,
        method=method,
        output_bounds=output_bounds_binary_variables,
        delta=delta,
        min_max_results_path_relaxed_global = min_max_results_path_relaxed_
    )

    return (explanation, model)

# Benchmark

## Utils

In [18]:
def export_milp_as_lp(mdl: mp.Model, file: str):
  mdl.export_as_lp(f"{file}")

In [19]:
def read_cplex_model(file: str):
  return cplex.Cplex(file)

In [20]:
def convert_string_to_pixel(pixel_str: str, matrix_size: tuple[int, int]) -> tuple[int, int]:
    # Remover o prefixo "x_"
    pixel_number = int(pixel_str.split("_")[1])
    # Obter as dimensões da matriz
    rows, cols = matrix_size
    # Calcular as coordenadas do pixel
    row_index = pixel_number // cols
    col_index = pixel_number % cols
    return row_index, col_index

In [21]:
def get_coordinates_from_explanation(
  explanation: list[LinearConstraint],
  matrix_size: tuple[int, int],
) -> list[tuple[int, int]]:
    coordinates = []
    for constraint in explanation:
        # Extrair coordenadas da variável associada à restrição linear
        variable_name = constraint.left_expr.name
        x, y = convert_string_to_pixel(variable_name, matrix_size)
        coordinates.append((x, y))
    return coordinates

In [22]:
def create_directory(path_to_create:str):
  array_splited = path_to_create.split('/')
  min_max_path_full = './'
  for path in array_splited:
    if path == '':
      continue
    min_max_path_full = f'{min_max_path_full}/{path}'
    if not os.path.exists(min_max_path_full):
      os.makedirs(min_max_path_full)
  return array_splited

In [23]:

# def benchmark_instance( 
#     model_h5_file:str,
#     initial_network: Any,
#     initial_network_relaxed: Any,
#     configurations: list,
#     model_h5: Any,
#     n_classes: int,
#     data: pd.DataFrame,
#     results: pd.DataFrame,
#     path_results: str,
#     instance_index: int,
#     file_results: str,
#     resultados: pd.DataFrame,
#     delta = 0.1,
#     use_milp_original=False,
#     matrix_size = (8, 8)):
#     (
#         tempo_original,
#         len_original,
#         explanation_original
#     ) = [None] * 3
    
    
    
    
#     min_max_results_path_original = f'{path_results}/min_max/instance_{instance_index}/original'
#     min_max_results_path_relaxed = f'{path_results}/min_max/instance_{instance_index}/relaxed'
#     min_max_results_path_relaxed_global = f'{path_results}/min_max/instance_{instance_index}/relaxed_global'
#     # create_directory(min_max_results_path_original)
#     # create_directory(min_max_results_path_relaxed)
#     # create_directory(min_max_results_path_relaxed_global)
#     predict = None 

#     if use_milp_original:
#         # explain_instance original
#         start_time = time()
#         (explanation_original, model_milp_original, predict_network_output) = explain_instance(
#             initial_network = initial_network,
#             configuration = configurations[0],
#             instance_index = instance_index,
#             data = data,
#             model_h5 = model_h5,
#             n_classes = n_classes,
#             min_max_results_path_original = min_max_results_path_original
#         )
#         predict = predict_network_output
#         end_time = time()
#         tempo_original = end_time - start_time
#         len_original = len(explanation_original)

#     # explain_instance_relaxed local
#     start_time = time()
#     (explanation_relaxed, model_milp_relaxed) = explain_instance_relaxed(
#         initial_network = initial_network,
#         initial_network_relaxed = initial_network_relaxed,
#         configuration = configurations[0],
#         instance_index = instance_index,
#         data = data,
#         model_h5 = model_h5,
#         n_classes = n_classes,
#         delta=delta,
#         min_max_results_path_relaxed_ = min_max_results_path_relaxed
#     )

#     end_time = time()
#     tempo_relaxado = end_time - start_time
#     len_relaxado = len(explanation_relaxed)

#     # explain_instance_relaxed global
#     start_time = time()
#     (explanation_relaxed_global, model_milp_relaxed_global) = explain_instance_relaxed(
#         initial_network = initial_network,
#         initial_network_relaxed = initial_network_relaxed,
#         configuration = configurations[0], #todo: modificar para passar o metodo diretamente
#         instance_index = instance_index,
#         data = data,
#         model_h5 = model_h5,
#         n_classes = n_classes,
#         delta=1,  # global
#         min_max_results_path_relaxed_ = min_max_results_path_relaxed_global
#     )

#     end_time = time()
#     tempo_relaxado_global = end_time - start_time
#     len_relaxado_global = len(explanation_relaxed_global)

#     len_resultados = len(resultados)
#     resultados.loc[len_resultados] = [
#         instance_index,
#         tempo_original,
#         tempo_relaxado,
#         tempo_relaxado_global,
#         len_original,
#         len_relaxado,
#         len_relaxado_global,
#         delta,
#         get_coordinates_from_explanation(explanation_original, matrix_size) if explanation_original is not None else None,
#         get_coordinates_from_explanation(explanation_relaxed, matrix_size),
#         get_coordinates_from_explanation(explanation_relaxed_global, matrix_size),
#     ]

#     # exportar modelos
#     if explanation_original is not None:
#         export_milp_as_lp(model_milp_original, f"{path_results}/original_after")
#     export_milp_as_lp(model_milp_relaxed, f"{path_results}/relaxed_after")

#     # salvar
#     resultados.to_csv(file_results, index=False)
#     return [explanation_original, explanation_relaxed, explanation_relaxed_global]

In [24]:


def benchmark_instance( 
    initial_network: Any,
    initial_network_relaxed: Any,
    configurations: list,
    model_h5: Any,
    n_classes: int,
    data: pd.DataFrame,
    instance_index: int,
    resultados: pd.DataFrame,
    file_result:str,
    delta = 0.1,
    matrix_size = (8, 8)):


    method = configurations[0]["method"]
    (
        mdl_milp_with_binary_variable,
        output_bounds_binary_variables,
        bounds,
    ) = initial_network
    
    network_input = data.iloc[instance_index, :-1]
    network_input = tf.reshape(tf.constant(network_input), (1, -1))
    network_output = model_h5.predict(tf.constant(network_input))[0]
    network_output = tf.argmax(network_output)
    
    # Relaxed
    model_milp_relaxed, output_bounds_relaxed = initial_network_relaxed
    mdl_aux_2 = model_milp_relaxed.clone()
    start_time = time()
    (explanation_relaxed_global, model_relaxed_global) = get_explanation_relaxed(
        mdl_aux_2,
        network_input,
        network_output,
        n_classes=n_classes,
        method=method,
        output_bounds=output_bounds_binary_variables,
        delta=1,
        min_max_results_path_relaxed_global = 'min_max_results_path_relaxed_'
    )
    end_time = time()
    time_relaxed_global = end_time - start_time
    len_relaxed_global = len(explanation_relaxed_global)

    if len_relaxed_global == 64:
        return
    
    # Original
    #mdl_aux = mdl_milp_with_binary_variable.clone()
    #start_time = time()
    #(explanation, model) = get_minimal_explanation(
    #    mdl_aux,
    #    network_input,
    #    network_output,
    #    n_classes=n_classes,
    #    method=method,
    #    output_bounds=output_bounds_binary_variables,
    #    min_max_results_path_original = 'min_max_results_path_original'
    #)
    #end_time = time()
    #tempo_original = end_time - start_time
    #len_original = len(explanation)
    #
    
    
    len_resultados = len(resultados)
    resultados.loc[len_resultados] = [
        instance_index,
        
        None, #tempo_original,
        None,
        time_relaxed_global,
        
        None, #len_original, 
        None,
        len_relaxed_global,
        
        delta,
        
        None, #get_coordinates_from_explanation(explanation, matrix_size) if explanation is not None else None,
        None,
        get_coordinates_from_explanation(explanation_relaxed_global, matrix_size),
        # explanation_relaxed_global, 
    ]
    resultados.to_csv(f'{file_result}/df.csv', index=False)

## Datasets

In [25]:
@dataclass
class Dataset:
    dir_path: str
    model: str
    n_classes: int

datasets: List[Dataset] = [
    Dataset(
        dir_path=f"{datasets_path}/digits",
        model="models/model_0layers_20neurons.h5",
        n_classes=10,
    ),
    Dataset(
        dir_path=f"{datasets_path}/digits",
        model="models/model_1layers_20neurons.h5",
        n_classes=10,
    ),
    Dataset(
        dir_path=f"{datasets_path}/digits",
        model="models/model_2layers_20neurons.h5",
        n_classes=10,
    ),
    Dataset(
        dir_path=f"{datasets_path}/digits",
        model="models/model_3layers_20neurons.h5",
        n_classes=10,
    ),
    Dataset(
        dir_path=f"{datasets_path}/digits",
        model="models/model_4layers_20neurons.h5",
        n_classes=10,
    ),
    Dataset(
        dir_path=f"{datasets_path}/digits",
        model="models/model_5layers_20neurons.h5",
        n_classes=10,
    ),
    Dataset(
        dir_path=f"{datasets_path}/iris",
        model="models/model_0layers_20neurons.h5",
        n_classes=3,
    ),
    Dataset(
        dir_path=f"{datasets_path}/iris",
        model="models/model_1layers_20neurons.h5",
        n_classes=3,
    ),
    Dataset(
        dir_path=f"{datasets_path}/iris",
        model="models/model_2layers_20neurons.h5",
        n_classes=3,
    ),
    Dataset(
        dir_path=f"{datasets_path}/iris",
        model="models/model_3layers_20neurons.h5",
        n_classes=3,
    ),
    Dataset(
        dir_path=f"{datasets_path}/iris",
        model="models/model_4layers_20neurons.h5",
        n_classes=3,
    ),
    Dataset(
        dir_path=f"{datasets_path}/iris",
        model="models/model_5layers_20neurons.h5",
        n_classes=3,
    ),
    Dataset(
        dir_path=f"{datasets_path}/iris",
        model="models/model_6layers_20neurons.h5",
        n_classes=3,
    ),
]

configurations = [{"method": "fischetti", "relaxe_constraints": True}]

## Configurações

In [26]:
def read_dataset(dir_path, model_h5_file):
  data_test = pd.read_csv(f"{dir_path}/test.csv")
  data_train = pd.read_csv(f"{dir_path}/train.csv")
  data = data_train._append(data_test)
  model_h5 = tf.keras.models.load_model(f"{dir_path}/{model_h5_file}")
  return (data, model_h5)

In [43]:

dataset_index = 9
matrix_size = (2, 2)
datasets[dataset_index]

Dataset(dir_path='.//datasets/iris', model='models/model_3layers_20neurons.h5', n_classes=3)

In [44]:
dir_path, n_classes, model_h5_file = (
    datasets[dataset_index].dir_path,
    datasets[dataset_index].n_classes,
    datasets[dataset_index].model,
)

configuration_index = 0
(data, model_h5) = read_dataset(dir_path, model_h5_file)

method = configurations[configuration_index]["method"]
relaxe_constraints = configurations[configuration_index]["relaxe_constraints"]

## Diretório dos resultados


In [45]:
def create_results_directory(dir_path:str, model_h5_file:str)->str:
  file_name_model = (f"{dir_path}/{model_h5_file}")
  file_name_model_result = (f"{dir_path}/results/{model_h5_file}")
  if not os.path.exists(file_name_model_result):
    os.makedirs(file_name_model_result)
  return file_name_model_result

In [46]:
file_result = create_results_directory(dir_path, model_h5_file)

## Modelos MILP

### Modelo MILP Original

In [47]:
initial_network = codify_network(model_h5, data, method, relaxe_constraints)
(
    mdl_milp_with_binary_variable,
    output_bounds_binary_variables,
    bounds,
) = initial_network

### Modelo MILP Relaxado

In [48]:
initial_network_relaxed = codify_network_relaxed(
    model_h5,
    mdl_milp_with_binary_variable,
    data,
    method,
    relaxe_constraints,
    output_bounds_binary_variables,
    bounds=bounds,
)

### Salvar Modelos MILPs


#### Salvar modelo MILP original


In [49]:
export_milp_as_lp(mdl_milp_with_binary_variable, f'{file_result}/original')

#### Salvar modelo MILP relaxado

In [50]:
(mdl_relaxed, output_bounds_relaxed) = initial_network_relaxed
export_milp_as_lp(mdl_relaxed, f'{file_result}/relaxed')

### Abrir Modelos MILPs

#### Abrir Modelo MILP Original

In [51]:
file_path_lp = f"{file_result}/original.lp"
# model_read = cplex.Cplex(caminho_do_arquivo_lp)
model_read = read_cplex_model(file_path_lp)

In [52]:
var_names = model_read.variables.get_names()
var_names_filtered = [nome for nome in var_names if nome.startswith('')]
print(var_names_filtered.__str__())


['x_0', 'x_1', 'x_2', 'x_3', 'y_0_0', 's_0_0', 'y_0_1', 's_0_1', 'y_0_2', 's_0_2', 'y_0_3', 's_0_3', 'y_0_4', 's_0_4', 'y_0_5', 's_0_5', 'y_0_6', 's_0_6', 'y_0_7', 's_0_7', 'y_0_8', 's_0_8', 'y_0_9', 's_0_9', 'y_0_10', 's_0_10', 'y_0_11', 's_0_11', 'y_0_12', 's_0_12', 'y_0_13', 's_0_13', 'y_0_14', 's_0_14', 'y_0_15', 's_0_15', 'y_0_16', 's_0_16', 'y_0_17', 's_0_17', 'y_0_18', 's_0_18', 'y_0_19', 's_0_19', 'y_1_0', 's_1_0', 'y_1_1', 's_1_1', 'y_1_2', 's_1_2', 'y_1_3', 's_1_3', 'y_1_4', 's_1_4', 'y_1_5', 's_1_5', 'y_1_6', 's_1_6', 'y_1_7', 's_1_7', 'y_1_8', 's_1_8', 'y_1_9', 's_1_9', 'y_1_10', 's_1_10', 'y_1_11', 's_1_11', 'y_1_12', 's_1_12', 'y_1_13', 's_1_13', 'y_1_14', 's_1_14', 'y_1_15', 's_1_15', 'y_1_16', 's_1_16', 'y_1_17', 's_1_17', 'y_1_18', 's_1_18', 'y_1_19', 's_1_19', 'y_2_0', 's_2_0', 'y_2_1', 's_2_1', 'y_2_2', 's_2_2', 'y_2_3', 's_2_3', 'y_2_4', 's_2_4', 'y_2_5', 's_2_5', 'y_2_6', 's_2_6', 'y_2_7', 's_2_7', 'y_2_8', 's_2_8', 'y_2_9', 's_2_9', 'y_2_10', 's_2_10', 'y_2_11', '

#### Abrir Modelo MILP Relaxado

In [53]:
path_file_lp_relaxed = f"{file_result}/relaxed.lp"
model_relaxed_read = read_cplex_model(path_file_lp_relaxed)

Warning, line 447: Name 's_0_4' does not exist.
Warning, line 453: Name 's_0_10' does not exist.
Warning, line 457: Name 's_0_14' does not exist.
Warning, line 458: Name 's_0_15' does not exist.
Warning, line 459: Name 's_0_16' does not exist.
Warning, line 467: Name 'a_0_4' does not exist.
Warning, line 473: Name 'a_0_10' does not exist.
Warning, line 477: Name 'a_0_14' does not exist.
Warning, line 478: Name 'a_0_15' does not exist.
Warning, line 479: Name 'a_0_16' does not exist.
Warning, line 507: Name 's_1_4' does not exist.
Warning, line 508: Name 's_1_5' does not exist.
Warning, line 511: Name 's_1_8' does not exist.
Warning, line 513: Name 's_1_10' does not exist.
Warning, line 514: Name 's_1_11' does not exist.
Warning, line 516: Name 's_1_13' does not exist.
Warning, line 517: Name 's_1_14' does not exist.
Warning, line 518: Name 's_1_15' does not exist.
Warning, line 527: Name 'a_1_4' does not exist.
Warning, line 528: Name 'a_1_5' does not exist.
18 name warnings not printe

In [54]:
var_names_relaxed = model_relaxed_read.variables.get_names()
var_names_relaxed_filtered = [nome for nome in var_names_relaxed if nome.startswith('')]
print(var_names_relaxed_filtered)

['x_0', 'x_1', 'x_2', 'x_3', 'y_0_0', 's_0_0', 'a_0_0', 'y_0_1', 's_0_1', 'a_0_1', 'y_0_2', 's_0_2', 'a_0_2', 'y_0_3', 's_0_3', 'a_0_3', 'y_0_4', 'y_0_5', 's_0_5', 'a_0_5', 'y_0_6', 's_0_6', 'a_0_6', 'y_0_7', 's_0_7', 'a_0_7', 'y_0_8', 's_0_8', 'a_0_8', 'y_0_9', 's_0_9', 'a_0_9', 'y_0_10', 'y_0_11', 's_0_11', 'a_0_11', 'y_0_12', 's_0_12', 'a_0_12', 'y_0_13', 's_0_13', 'a_0_13', 'y_0_14', 'y_0_15', 'y_0_16', 'y_0_17', 's_0_17', 'a_0_17', 'y_0_18', 's_0_18', 'a_0_18', 'y_0_19', 's_0_19', 'a_0_19', 'y_1_0', 's_1_0', 'a_1_0', 'y_1_1', 's_1_1', 'a_1_1', 'y_1_2', 's_1_2', 'a_1_2', 'y_1_3', 's_1_3', 'a_1_3', 'y_1_4', 'y_1_5', 'y_1_6', 's_1_6', 'a_1_6', 'y_1_7', 's_1_7', 'a_1_7', 'y_1_8', 'y_1_9', 's_1_9', 'a_1_9', 'y_1_10', 'y_1_11', 'y_1_12', 's_1_12', 'a_1_12', 'y_1_13', 'y_1_14', 'y_1_15', 'y_1_16', 's_1_16', 'a_1_16', 'y_1_17', 's_1_17', 'a_1_17', 'y_1_18', 's_1_18', 'a_1_18', 'y_1_19', 's_1_19', 'a_1_19', 'y_2_0', 's_2_0', 'a_2_0', 'y_2_1', 's_2_1', 'a_2_1', 'y_2_2', 's_2_2', 'a_2_2', 'y

## Executar Benchmark

In [55]:
# @title criar dataframe dos resultados

path_results = f"{file_result}"
if not os.path.exists(path_results):
    os.makedirs(path_results)
file_results = f"{path_results}/df.csv"
if os.path.exists(file_results):
    resultados = pd.read_csv(file_results)
else:
    resultados = pd.DataFrame(
        columns=[
            "instance_index",
            "tempo_original",
            "tempo_relaxado",
            "tempo_relaxado_global",
            "len_original",
            "len_relaxado",
            "len_relaxado_global",
            "delta",
            "explanation",
            "explanation_relaxed",
            "explanation_relaxed_global",
        ]
    )

In [56]:
minimo = 0
quantidade = len(data)
maximo = minimo + quantidade

In [57]:

delta = 0.1
for index in range(minimo, maximo):
  print(f"index: {index}")
  benchmark_instance(
    data = data,
    instance_index = index,
    delta = delta,
    matrix_size = matrix_size,
    configurations=configurations,
    model_h5=model_h5,
    n_classes=n_classes,
    initial_network=initial_network,
    initial_network_relaxed=initial_network_relaxed,
    resultados=resultados,
    file_result=file_result,
  )

minimo = maximo

index: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step    
index: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
index: 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
index: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 442ms/step
index: 4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
index: 5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
index: 6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
index: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
index: 8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
index: 9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
index: 10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
index: 11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
index: 12
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
index: 13
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
index: 14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
index: 15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
index: 16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
index: 17
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
index: 18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
index: 19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
index: 20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s